In [1]:
import cv2
import time
from ultralytics import YOLO
import requests
import random




In [2]:
# ==============================
# CONFIGURATION
# ==============================
STREAM_URL = "http://172.18.230.94:8080/video"
MODEL_PATH = "best.pt"
CONF_THRES = 0.4

# Fixed zones (xmin, ymin, xmax, ymax)
ZONE_1 = (513, 243, 1200, 928)
ZONE_2 = (1200, 242, 1924, 952)

# Timing & filtering
FRAME_CONFIRM = 5          # frames required to confirm presence
FRAME_RELEASE = 5          # frames required to confirm absence
THEFT_CONFIRM_FRAMES = 3   # frames to confirm theft
MAX_DIM_TIME = 5.0         # seconds (cap)
ALERT_COOLDOWN = 30        # seconds
ROBBERY_DURATION = 4.0     # seconds

# Raspberry Pi Configuration
PI_IP = "10.197.62.105"   # Change this to your Pi's IP
PORT = 8000
BASE_URL = f"http://{PI_IP}:{PORT}"

# Pushover Configuration
USER_KEY = "YOUR_USER_KEY"
APP_TOKEN = "YOUR_APP_TOKEN"

In [3]:
model = YOLO(MODEL_PATH)
print("✅ Model loaded")
print("Classes:", model.names)


✅ Model loaded
Classes: {0: 'imp_item', 1: 'imp_item_stolen', 2: 'person_1', 3: 'person_2'}


In [4]:
zone_state = {
    1: {
        "state": "ABSENT",
        "present_frames": 0,
        "absent_frames": 0,
        "enter_time": None
    },
    2: {
        "state": "ABSENT",
        "present_frames": 0,
        "absent_frames": 0,
        "enter_time": None
    }
}

theft_counter = {1: 0, 2: 0}
last_alert_time = 0


In [5]:

# ==============================
# RASPBERRY PI LED CONTROL
# ==============================
def led_on(led_id: int):
    """Turn LED on"""
    try:
        url = f"{BASE_URL}/led/{led_id}/on"
        r = requests.post(url, timeout=2)
        print(f"💡 LED {led_id} ON ->", r.json())
    except Exception as e:
        print(f"❌ LED {led_id} ON failed:", e)

def led_fade_off(led_id: int, duration: float):
    """Fade LED off over duration"""
    try:
        url = f"{BASE_URL}/led/{led_id}/off"
        params = {"duration": duration}
        r = requests.post(url, params=params, timeout=2)
        print(f"🌗 LED {led_id} FADE OFF ({duration:.2f}s) ->", r.json())
    except Exception as e:
        print(f"❌ LED {led_id} FADE OFF failed:", e)

def start_robbery(glow: int, flicker: int):
    """Start robbery mode - glow one LED, flicker the other"""
    try:
        url = f"{BASE_URL}/robbery/start"
        params = {"glow": glow, "flicker": flicker}
        r = requests.post(url, params=params, timeout=2)
        print(f"🚨 ROBBERY MODE START (glow={glow}, flicker={flicker}) ->", r.json())
    except Exception as e:
        print(f"❌ ROBBERY START failed:", e)

def stop_robbery():
    """Stop robbery mode"""
    try:
        url = f"{BASE_URL}/robbery/stop"
        r = requests.post(url, timeout=2)
        print(f"✅ ROBBERY MODE STOP ->", r.json())
    except Exception as e:
        print(f"❌ ROBBERY STOP failed:", e)

In [6]:

# ==============================
# LED CONTROL WRAPPERS
# ==============================
def turn_led_on(zone):
    """Turn on LED for specific zone"""
    print(f"💡 LED {zone} ON")
    led_on(zone)

def dim_led(zone, duration):
    """Dim LED over specified duration"""
    print(f"🌗 LED {zone} DIM over {duration:.2f}s")
    led_fade_off(zone, duration)

def blink_led(zone):
    """Start blinking LED for theft alert"""
    print(f"🚨 LED {zone} BLINKING")
    # Determine which LED glows and which flickers
    # The zone where theft occurred stays on (glows)
    # The other zone flickers
    other_zone = 2 if zone == 1 else 1
    start_robbery(glow=zone, flicker=other_zone)

def stop_all_leds():
    """Stop all LED activity"""
    print("🛑 Stop all LEDs")
    stop_robbery()

In [7]:
# ==============================
# PUSHOVER ALERT
# ==============================
def send_burglary_alert(msg):
    """Send push notification for burglary"""
    global last_alert_time
    now = time.time()

    if now - last_alert_time < ALERT_COOLDOWN:
        print("⏳ Alert on cooldown, skipping...")
        return

    try:
        requests.post(
            "https://api.pushover.net/1/messages.json",
            data={
                "token": APP_TOKEN,
                "user": USER_KEY,
                "title": "Security Alert",
                "message": msg,
                "priority": 1
            },
            timeout=5
        )
        last_alert_time = now
        print("📱 Alert sent:", msg)
    except Exception as e:
        print(f"❌ Alert failed: {e}")

In [8]:
# ==============================
# ZONE DETECTION
# ==============================
def inside_zone(cx, cy, zone):
    """Check if point (cx, cy) is inside zone"""
    x1, y1, x2, y2 = zone
    return x1 <= cx <= x2 and y1 <= cy <= y2

# ==============================
# EVENT HANDLERS
# ==============================
def handle_entry(zone_id):
    """Handle person entering zone"""
    z = zone_state[zone_id]
    z["state"] = "PRESENT"
    z["enter_time"] = time.time()
    turn_led_on(zone_id)

def handle_exit(zone_id):
    """Handle person exiting zone"""
    z = zone_state[zone_id]
    stay = time.time() - z["enter_time"]
    dim_time = min(stay, MAX_DIM_TIME)
    z["state"] = "ABSENT"
    dim_led(zone_id, dim_time)

def handle_theft(zone_id):
    """Handle theft detection"""
    global robbery_active, robbery_zone, robbery_start_time

    # Latch: ignore repeated detections
    if robbery_active:
        return

    robbery_active = True
    robbery_zone = zone_id
    robbery_start_time = time.time()

    blink_led(zone_id)
    send_burglary_alert(f"🚨 Theft detected in Zone {zone_id}")

    print(f"🚨 ROBBERY ACTIVE in Zone {zone_id}")



In [9]:
# ==============================
# MAIN LOOP
# ==============================
def main():
    global robbery_active, robbery_zone, robbery_start_time

    # Load model
    model = YOLO(MODEL_PATH)
    print("✅ Model loaded")
    print("Classes:", model.names)

    # Generate colors for classes
    num_classes = len(model.names)
    class_colors = generate_colors(num_classes)
    print(f"✅ Generated {num_classes} colors for classes")

    # Open video stream
    cap = cv2.VideoCapture(STREAM_URL)
    if not cap.isOpened():
        raise RuntimeError("❌ Camera stream failed")

    cv2.namedWindow("Feed", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Feed", 960, 540)

    print("✅ Stream started")
    print("Press 'q' to quit")

    try:
        while True:
            # ---- AUTO END ROBBERY AFTER FIXED TIME ----
            if robbery_active and (time.time() - robbery_start_time >= ROBBERY_DURATION):
                robbery_active = False
                robbery_zone = None
                robbery_start_time = None
                stop_all_leds()
                print("✅ Robbery auto-ended after timeout")

            ret, frame = cap.read()
            if not ret:
                print("⚠️ Frame read failed, retrying...")
                continue

            # Per-frame presence
            person_detected = {1: False, 2: False}

            # Run YOLO inference
            results = model(frame, conf=CONF_THRES, verbose=False)

            for r in results:
                if r.boxes is None:
                    continue

                for box in r.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cls = int(box.cls[0])
                    conf = float(box.conf[0])
                    label = model.names[cls]

                    cx = (x1 + x2) // 2
                    cy = (y1 + y2) // 2

                    # Get color for this class
                    color = class_colors[cls]

                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)

                    # Draw label with background
                    text_label = f"{label} {conf:.2f}"
                    (label_w, label_h), baseline = cv2.getTextSize(
                        text_label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2
                    )

                    # Background rectangle
                    cv2.rectangle(
                        frame,
                        (x1, y1 - label_h - baseline - 5),
                        (x1 + label_w, y1),
                        color,
                        -1
                    )

                    # White text
                    cv2.putText(
                        frame,
                        text_label,
                        (x1, y1 - baseline - 5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,
                        (255, 255, 255),
                        2
                    )

                    # PERSON LOGIC
                    if label in ["person_1", "person_2"]:
                        if inside_zone(cx, cy, ZONE_1):
                            person_detected[1] = True
                        elif inside_zone(cx, cy, ZONE_2):
                            person_detected[2] = True

                    # THEFT LOGIC
                    if label == "imp_item_stolen":
                        for zid, zone in [(1, ZONE_1), (2, ZONE_2)]:
                            if inside_zone(cx, cy, zone):
                                theft_counter[zid] += 1
                                if theft_counter[zid] >= THEFT_CONFIRM_FRAMES:
                                    handle_theft(zid)
                                    theft_counter[zid] = 0
                    else:
                        # Reset theft counter if no theft detected
                        for zid in [1, 2]:
                            theft_counter[zid] = 0

            # STATE MACHINE UPDATE
            for zid in [1, 2]:
                z = zone_state[zid]

                if person_detected[zid]:
                    z["present_frames"] += 1
                    z["absent_frames"] = 0

                    if z["present_frames"] >= FRAME_CONFIRM and z["state"] == "ABSENT":
                        handle_entry(zid)

                else:
                    z["absent_frames"] += 1
                    z["present_frames"] = 0

                    if z["absent_frames"] >= FRAME_RELEASE and z["state"] == "PRESENT":
                        handle_exit(zid)

            # Draw fixed zones
            cv2.rectangle(frame, ZONE_1[:2], ZONE_1[2:], (0, 0, 255), 3)
            cv2.putText(frame, "ZONE 1", (ZONE_1[0] + 10, ZONE_1[1] + 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.rectangle(frame, ZONE_2[:2], ZONE_2[2:], (255, 0, 0), 3)
            cv2.putText(frame, "ZONE 2", (ZONE_2[0] + 10, ZONE_2[1] + 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            # Show robbery status
            if robbery_active:
                cv2.putText(frame, f"🚨 ROBBERY ACTIVE - ZONE {robbery_zone}",
                           (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

            cv2.imshow("Feed", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()
        stop_all_leds()  # Clean shutdown
        print("🛑 Shutdown complete")

if __name__ == "__main__":
    main()

✅ Model loaded
Classes: {0: 'imp_item', 1: 'imp_item_stolen', 2: 'person_1', 3: 'person_2'}


NameError: name 'generate_colors' is not defined

In [ ]:
if __name__ == "__main__":
    main()

NameError: name 'led_fade_off' is not defined

In [ ]:
import cv2
import time
from ultralytics import YOLO
import requests
import random

# ==============================
# CONFIGURATION
# ==============================
STREAM_URL = "http://172.18.230.94:8080/video"
MODEL_PATH = "best.pt"
CONF_THRES = 0.4

# Fixed zones (xmin, ymin, xmax, ymax)
ZONE_1 = (513, 243, 1200, 928)
ZONE_2 = (1200, 242, 1924, 952)

# Timing & filtering
FRAME_CONFIRM = 5          # frames required to confirm presence
FRAME_RELEASE = 5          # frames required to confirm absence
THEFT_CONFIRM_FRAMES = 3   # frames to confirm theft
MAX_DIM_TIME = 5.0         # seconds (cap)
ALERT_COOLDOWN = 30        # seconds
ROBBERY_DURATION = 4.0     # seconds

# Raspberry Pi Configuration
PI_IP = "10.197.62.105"   # Change this to your Pi's IP
PORT = 8000
BASE_URL = f"http://{PI_IP}:{PORT}"

# Pushover Configuration
USER_KEY = "YOUR_USER_KEY"
APP_TOKEN = "YOUR_APP_TOKEN"

# ==============================
# GLOBAL STATE
# ==============================
robbery_active = False
robbery_zone = None
robbery_start_time = None
last_alert_time = 0

zone_state = {
    1: {
        "state": "ABSENT",
        "present_frames": 0,
        "absent_frames": 0,
        "enter_time": None
    },
    2: {
        "state": "ABSENT",
        "present_frames": 0,
        "absent_frames": 0,
        "enter_time": None
    }
}

theft_counter = {1: 0, 2: 0}

# ==============================
# COLOR GENERATION FOR CLASSES
# ==============================
def generate_colors(num_classes):
    """Generate distinct colors for each class"""
    random.seed(42)
    colors = []
    for i in range(num_classes):
        colors.append((
            random.randint(50, 255),
            random.randint(50, 255),
            random.randint(50, 255)
        ))
    return colors

# ==============================
# RASPBERRY PI LED CONTROL
# ==============================
def led_on(led_id: int):
    """Turn LED on"""
    try:
        url = f"{BASE_URL}/led/{led_id}/on"
        r = requests.post(url, timeout=2)
        print(f"💡 LED {led_id} ON ->", r.json())
    except Exception as e:
        print(f"❌ LED {led_id} ON failed:", e)

def led_fade_off(led_id: int, duration: float):
    """Fade LED off over duration"""
    try:
        url = f"{BASE_URL}/led/{led_id}/off"
        params = {"duration": duration}
        r = requests.post(url, params=params, timeout=2)
        print(f"🌗 LED {led_id} FADE OFF ({duration:.2f}s) ->", r.json())
    except Exception as e:
        print(f"❌ LED {led_id} FADE OFF failed:", e)

def start_robbery(glow: int, flicker: int):
    """Start robbery mode - glow one LED, flicker the other"""
    try:
        url = f"{BASE_URL}/robbery/start"
        params = {"glow": glow, "flicker": flicker}
        r = requests.post(url, params=params, timeout=2)
        print(f"🚨 ROBBERY MODE START (glow={glow}, flicker={flicker}) ->", r.json())
    except Exception as e:
        print(f"❌ ROBBERY START failed:", e)

def stop_robbery():
    """Stop robbery mode"""
    try:
        url = f"{BASE_URL}/robbery/stop"
        r = requests.post(url, timeout=2)
        print(f"✅ ROBBERY MODE STOP ->", r.json())
    except Exception as e:
        print(f"❌ ROBBERY STOP failed:", e)

# ==============================
# LED CONTROL WRAPPERS
# ==============================
def turn_led_on(zone):
    """Turn on LED for specific zone"""
    print(f"💡 LED {zone} ON")
    led_on(zone)

def dim_led(zone, duration):
    """Dim LED over specified duration"""
    print(f"🌗 LED {zone} DIM over {duration:.2f}s")
    led_fade_off(zone, duration)

def blink_led(zone):
    """Start blinking LED for theft alert"""
    print(f"🚨 LED {zone} BLINKING")
    # Determine which LED glows and which flickers
    # The zone where theft occurred stays on (glows)
    # The other zone flickers
    other_zone = 2 if zone == 1 else 1
    start_robbery(glow=zone, flicker=other_zone)

def stop_all_leds():
    """Stop all LED activity"""
    print("🛑 Stop all LEDs")
    stop_robbery()

# ==============================
# PUSHOVER ALERT
# ==============================
def send_burglary_alert(msg):
    """Send push notification for burglary"""
    global last_alert_time
    now = time.time()

    if now - last_alert_time < ALERT_COOLDOWN:
        print("⏳ Alert on cooldown, skipping...")
        return

    try:
        requests.post(
            "https://api.pushover.net/1/messages.json",
            data={
                "token": APP_TOKEN,
                "user": USER_KEY,
                "title": "Security Alert",
                "message": msg,
                "priority": 1
            },
            timeout=5
        )
        last_alert_time = now
        print("📱 Alert sent:", msg)
    except Exception as e:
        print(f"❌ Alert failed: {e}")

# ==============================
# ZONE DETECTION
# ==============================
def inside_zone(cx, cy, zone):
    """Check if point (cx, cy) is inside zone"""
    x1, y1, x2, y2 = zone
    return x1 <= cx <= x2 and y1 <= cy <= y2

# ==============================
# EVENT HANDLERS
# ==============================
def handle_entry(zone_id):
    """Handle person entering zone"""
    z = zone_state[zone_id]
    z["state"] = "PRESENT"
    z["enter_time"] = time.time()
    turn_led_on(zone_id)

def handle_exit(zone_id):
    """Handle person exiting zone"""
    z = zone_state[zone_id]
    stay = time.time() - z["enter_time"]
    dim_time = min(stay, MAX_DIM_TIME)
    z["state"] = "ABSENT"
    dim_led(zone_id, dim_time)

def handle_theft(zone_id):
    """Handle theft detection"""
    global robbery_active, robbery_zone, robbery_start_time

    # Latch: ignore repeated detections
    if robbery_active:
        return

    robbery_active = True
    robbery_zone = zone_id
    robbery_start_time = time.time()

    blink_led(zone_id)
    send_burglary_alert(f"🚨 Theft detected in Zone {zone_id}")

    print(f"🚨 ROBBERY ACTIVE in Zone {zone_id}")

# ==============================
# MAIN LOOP
# ==============================
def main():
    global robbery_active, robbery_zone, robbery_start_time

    # Load model
    model = YOLO(MODEL_PATH)
    print("✅ Model loaded")
    print("Classes:", model.names)

    # Generate colors for classes
    num_classes = len(model.names)
    class_colors = generate_colors(num_classes)
    print(f"✅ Generated {num_classes} colors for classes")

    # Open video stream
    cap = cv2.VideoCapture(STREAM_URL)
    if not cap.isOpened():
        raise RuntimeError("❌ Camera stream failed")

    cv2.namedWindow("Feed", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Feed", 960, 540)

    print("✅ Stream started")
    print("Press 'q' to quit")

    try:
        while True:
            # ---- AUTO END ROBBERY AFTER FIXED TIME ----
            if robbery_active and (time.time() - robbery_start_time >= ROBBERY_DURATION):
                robbery_active = False
                robbery_zone = None
                robbery_start_time = None
                stop_all_leds()
                print("✅ Robbery auto-ended after timeout")

            ret, frame = cap.read()
            if not ret:
                print("⚠️ Frame read failed, retrying...")
                continue

            # Per-frame presence
            person_detected = {1: False, 2: False}

            # Run YOLO inference
            results = model(frame, conf=CONF_THRES, verbose=False)

            # Track which zones have theft detected this frame
            theft_detected_in_zone = {1: False, 2: False}
            
            for r in results:
                if r.boxes is None:
                    continue

                for box in r.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cls = int(box.cls[0])
                    conf = float(box.conf[0])
                    label = model.names[cls]

                    cx = (x1 + x2) // 2
                    cy = (y1 + y2) // 2

                    # Get color for this class
                    color = class_colors[cls]

                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)

                    # Draw label with background
                    text_label = f"{label} {conf:.2f}"
                    (label_w, label_h), baseline = cv2.getTextSize(
                        text_label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2
                    )

                    # Background rectangle
                    cv2.rectangle(
                        frame,
                        (x1, y1 - label_h - baseline - 5),
                        (x1 + label_w, y1),
                        color,
                        -1
                    )

                    # White text
                    cv2.putText(
                        frame,
                        text_label,
                        (x1, y1 - baseline - 5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,
                        (255, 255, 255),
                        2
                    )

                    # PERSON LOGIC
                    if label in ["person_1", "person_2"]:
                        if inside_zone(cx, cy, ZONE_1):
                            person_detected[1] = True
                        elif inside_zone(cx, cy, ZONE_2):
                            person_detected[2] = True

                    # THEFT LOGIC
                    if label == "imp_item_stolen":
                        for zid, zone in [(1, ZONE_1), (2, ZONE_2)]:
                            if inside_zone(cx, cy, zone):
                                theft_detected_in_zone[zid] = True
                                theft_counter[zid] += 1
                                if theft_counter[zid] >= THEFT_CONFIRM_FRAMES:
                                    handle_theft(zid)
                                    theft_counter[zid] = 0
            
            # Reset theft counters ONLY for zones where no theft was detected this frame
            for zid in [1, 2]:
                if not theft_detected_in_zone[zid]:
                    theft_counter[zid] = 0

            # STATE MACHINE UPDATE
            for zid in [1, 2]:
                z = zone_state[zid]

                if person_detected[zid]:
                    z["present_frames"] += 1
                    z["absent_frames"] = 0

                    if z["present_frames"] >= FRAME_CONFIRM and z["state"] == "ABSENT":
                        handle_entry(zid)

                else:
                    z["absent_frames"] += 1
                    z["present_frames"] = 0

                    if z["absent_frames"] >= FRAME_RELEASE and z["state"] == "PRESENT":
                        handle_exit(zid)

            # Draw fixed zones
            cv2.rectangle(frame, ZONE_1[:2], ZONE_1[2:], (0, 0, 255), 3)
            cv2.putText(frame, "ZONE 1", (ZONE_1[0] + 10, ZONE_1[1] + 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.rectangle(frame, ZONE_2[:2], ZONE_2[2:], (255, 0, 0), 3)
            cv2.putText(frame, "ZONE 2", (ZONE_2[0] + 10, ZONE_2[1] + 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            # Show robbery status
            if robbery_active:
                cv2.putText(frame, f"🚨 ROBBERY ACTIVE - ZONE {robbery_zone}",
                           (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

            cv2.imshow("Feed", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()
        stop_all_leds()  # Clean shutdown
        print("🛑 Shutdown complete")

if __name__ == "__main__":
    main()

✅ Model loaded
Classes: {0: 'imp_item', 1: 'imp_item_stolen', 2: 'person_1', 3: 'person_2'}
✅ Generated 4 colors for classes
✅ Stream started
Press 'q' to quit
💡 LED 1 ON
💡 LED 1 ON -> {'status': 'ON', 'led': 1}
🌗 LED 1 DIM over 3.00s
🌗 LED 1 FADE OFF (3.00s) -> {'status': 'FADING', 'led': 1, 'duration_seconds': 2.996830940246582}
💡 LED 1 ON
💡 LED 1 ON -> {'status': 'ON', 'led': 1}
🌗 LED 1 DIM over 4.23s
🌗 LED 1 FADE OFF (4.23s) -> {'status': 'FADING', 'led': 1, 'duration_seconds': 4.233761787414551}
💡 LED 1 ON
💡 LED 1 ON -> {'status': 'ON', 'led': 1}
🌗 LED 1 DIM over 4.80s
🌗 LED 1 FADE OFF (4.80s) -> {'status': 'FADING', 'led': 1, 'duration_seconds': 4.796816825866699}
💡 LED 1 ON
💡 LED 1 ON -> {'status': 'ON', 'led': 1}


[mjpeg @ 0x15337dc80] overread 8


🌗 LED 1 DIM over 3.42s
🌗 LED 1 FADE OFF (3.42s) -> {'status': 'FADING', 'led': 1, 'duration_seconds': 3.4219162464141846}
💡 LED 1 ON
💡 LED 1 ON -> {'status': 'ON', 'led': 1}
🌗 LED 1 DIM over 5.00s
🌗 LED 1 FADE OFF (5.00s) -> {'status': 'FADING', 'led': 1, 'duration_seconds': 5.0}
